In [0]:
-- Create SQL Procedure for ADS_RDS_EVENT_TYPES_MAIN ETL Process
CREATE OR REPLACE PROCEDURE gap_catalog.ads_etl_owner.sp_ADS_RDS_EVENT_TYPES_MAIN(
    p_load_date STRING,
    p_process_key STRING
)
LANGUAGE SQL
 SQL SECURITY INVOKER
COMMENT 'ETL procedure for ADS_RDS_EVENT_TYPES_MAIN - converted from notebook format'
AS
BEGIN
    -- Declare local variables
    DECLARE dif_table_name STRING DEFAULT 'gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_MAIN_ADS_MAP_SCD_DIFF';
    DECLARE my_log_id STRING;
    BEGIN
        CALL gap_catalog.log.start_performance_test(
            test_case_name => 'ADS_RDS_PROC',
            step_name => 'ADS_RDS_EVENT_TYPES_MAIN',
            psource_sys_origin  => 'RDS_ANALYTICALEVENTTYPES',
            process_key => p_process_key,
            target_table => 'EVENT_TYPES',
            log_id => my_log_id,
            p_load_date => p_load_date
        );

        /* Truncate XC Table */
        TRUNCATE TABLE gap_catalog.ads_etl_owner.XC_RDS_EVENT_TYPES_MAIN;

        /* Fill XC Table */
        INSERT INTO gap_catalog.ads_etl_owner.XC_RDS_EVENT_TYPES_MAIN 
              with C_0FILTERA5N9CFO3HGCAMJOS6QL7KQ5MRB as (
              select
                  RDS_ANALYTICALEVENTTYPES_1.id  ID,
                  RDS_ANALYTICALEVENTTYPES_1.eventtablename  EVENTTABLENAME,
                  RDS_ANALYTICALEVENTTYPES_1.sourcesystem  SOURCESYSTEM,
                  RDS_ANALYTICALEVENTTYPES_1.value  VALUE,
                  RDS_ANALYTICALEVENTTYPES_1.data_path  DATA_PATH,
                  RDS_ANALYTICALEVENTTYPES_1.target  TARGET
                from gap_catalog.ads_etl_owner.DLK_ADS_LOV_RDS_ANALYTICALEVENTTYPES RDS_ANALYTICALEVENTTYPES_1
                where ( RDS_ANALYTICALEVENTTYPES_1.sys = 'Brasil'
                    and RDS_ANALYTICALEVENTTYPES_1.lang  = 'CZ'
                    and CAST(from_utc_timestamp(SYS_EFFECTIVE_DATE, 'Europe/Prague') AS DATE) = CAST(p_load_date AS DATE)
                      )
              )
              select  /*+no hint*/
                      /*seznam vkladanych nebo updatovanych sloupcu bez SK, auditnich atribudu a deleted flagu*/
                  FILTER_A.ID as EVETP_SOURCE_ID,
                  nvl(FILTER_A.SOURCESYSTEM,'ADS') as EVETP_SOURCE_SYSTEM_ID,
                  'RDS_ANALYTICALEVENTTYPES' as EVETP_SOURCE_SYS_ORIGIN,
                  FILTER_A.VALUE as EVETP_DESC,
                  FILTER_A.EVENTTABLENAME as EVETP_EVENT_TABLE_NAME,
                  nvl(FILTER_A.TARGET,'XNA') as EVETP_TARGET,
                  nvl(FILTER_A.DATA_PATH,'XNA') as EVETP_DATA_PATH,
                  'N' as EVETP_DELETED_FLAG
                from (C_0FILTERA5N9CFO3HGCAMJOS6QL7KQ5MRB FILTER_A)
                where (1=1)
        ;

        /* Cleanup DIFF Table */
        DROP TABLE IF EXISTS gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_MAIN_ADS_MAP_SCD_DIFF;

        /* Create DIFF Table */
        create  table gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_MAIN_ADS_MAP_SCD_DIFF
        ( tech_del_flg  char(1),
          tech_new_rec  char(1),
          tech_rid      varchar(255),
          EVETP_KEY  decimal(38,0),
          EVETP_KEY_NEW BIGINT GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),
          EVETP_SOURCE_ID  varchar(400),
          EVETP_SOURCE_SYSTEM_ID  varchar(120),
          EVETP_SOURCE_SYS_ORIGIN  varchar(120),
          EVETP_DESC  varchar(4000),
          EVETP_EVENT_TABLE_NAME  varchar(120),
          EVETP_TARGET  varchar(120),
          EVETP_DATA_PATH  varchar(120))
        ;

        /* Populate DIFF Table - New/Updated Records */
        insert into gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_MAIN_ADS_MAP_SCD_DIFF
        (
          tech_del_flg,
          tech_new_rec,
          tech_rid,
          EVETP_KEY,
          EVETP_SOURCE_ID,
          EVETP_SOURCE_SYSTEM_ID,
          EVETP_SOURCE_SYS_ORIGIN,
          EVETP_DESC,
          EVETP_EVENT_TABLE_NAME,
          EVETP_TARGET,
          EVETP_DATA_PATH
        )
        select   'N' as tech_del_flg, 
            case when trg.EVETP_SOURCE_ID is null then 'Y' else 'N' end as tech_new_rec,
            trg.rid as tech_rid,
           trg.EVETP_KEY, 
           src.EVETP_SOURCE_ID, 
           src.EVETP_SOURCE_SYSTEM_ID, 
           src.EVETP_SOURCE_SYS_ORIGIN, 
           src.EVETP_DESC, 
           src.EVETP_EVENT_TABLE_NAME, 
           src.EVETP_TARGET, 
           src.EVETP_DATA_PATH
         from 
            (select /*+ full(xc) */      EVETP_SOURCE_ID, 
               EVETP_SOURCE_SYSTEM_ID, 
               EVETP_SOURCE_SYS_ORIGIN, 
               EVETP_DESC, 
               EVETP_EVENT_TABLE_NAME, 
               EVETP_DELETED_FLAG, 
               EVETP_TARGET, 
               EVETP_DATA_PATH
               from gap_catalog.ads_etl_owner.XC_RDS_EVENT_TYPES_MAIN xc
            where (EVETP_SOURCE_SYS_ORIGIN='RDS_ANALYTICALEVENTTYPES')) src LEFT JOIN
            (select  evetp_key||'.'||evetp_valid_from as rid, t.* from gap_catalog.ads_owner.EVENT_TYPES t
              where EVETP_CURRENT_FLAG  = 'Y'
                and EVETP_VALID_TO  = DATE('3000-01-01')
               and (EVETP_SOURCE_SYS_ORIGIN='RDS_ANALYTICALEVENTTYPES')      ) trg
        ON trg.EVETP_SOURCE_ID = src.EVETP_SOURCE_ID
         and trg.EVETP_SOURCE_SYSTEM_ID = src.EVETP_SOURCE_SYSTEM_ID
         and trg.EVETP_SOURCE_SYS_ORIGIN = src.EVETP_SOURCE_SYS_ORIGIN
         and trg.EVETP_VALID_TO = DATE('3000-01-01') WHERE (
             decode( src.EVETP_DESC,trg.EVETP_DESC,1,0 ) = 0  or
             decode( src.EVETP_EVENT_TABLE_NAME,trg.EVETP_EVENT_TABLE_NAME,1,0 ) = 0  or
             decode( src.EVETP_DELETED_FLAG,trg.EVETP_DELETED_FLAG,1,0 ) = 0  or
             decode( src.EVETP_TARGET,trg.EVETP_TARGET,1,0 ) = 0  or
             decode( src.EVETP_DATA_PATH,trg.EVETP_DATA_PATH,1,0 ) = 0 or 
             trg.EVETP_SOURCE_ID is null   );

        /* Populate DIFF Table - Deleted Records */
        insert into gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_MAIN_ADS_MAP_SCD_DIFF
        (
          tech_del_flg,
          tech_new_rec,
          tech_rid,
          EVETP_KEY,
          EVETP_SOURCE_ID,
          EVETP_SOURCE_SYSTEM_ID,
          EVETP_SOURCE_SYS_ORIGIN,
          EVETP_DESC,
          EVETP_EVENT_TABLE_NAME,
          EVETP_TARGET,
          EVETP_DATA_PATH
        )
        select   'Y' as tech_del_flg, 
         'N' as tech_new_rec, 
         trg.rid as tech_rid, 
           trg.EVETP_KEY, 
           trg.EVETP_SOURCE_ID, 
           trg.EVETP_SOURCE_SYSTEM_ID, 
           trg.EVETP_SOURCE_SYS_ORIGIN, 
           trg.EVETP_DESC, 
           trg.EVETP_EVENT_TABLE_NAME, 
           trg.EVETP_TARGET, 
           trg.EVETP_DATA_PATH
         from 
            (select  evetp_key||'.'||evetp_valid_from as rid, t.* from gap_catalog.ads_owner.EVENT_TYPES t
              where EVETP_CURRENT_FLAG  = 'Y' and EVETP_DELETED_FLAG  = 'N'
                and EVETP_VALID_TO  = DATE('3000-01-01')
               and (EVETP_SOURCE_SYS_ORIGIN='RDS_ANALYTICALEVENTTYPES')      ) trg LEFT JOIN
            (select /*+ full(xc) */      EVETP_SOURCE_ID, 
               EVETP_SOURCE_SYSTEM_ID, 
               EVETP_SOURCE_SYS_ORIGIN, 
               DATE('3000-01-01') as EVETP_VALID_TO
               from gap_catalog.ads_etl_owner.XC_RDS_EVENT_TYPES_MAIN xc
            where (EVETP_SOURCE_SYS_ORIGIN='RDS_ANALYTICALEVENTTYPES')) src
        ON trg.EVETP_SOURCE_ID = src.EVETP_SOURCE_ID
         and trg.EVETP_SOURCE_SYSTEM_ID = src.EVETP_SOURCE_SYSTEM_ID
         and trg.EVETP_SOURCE_SYS_ORIGIN = src.EVETP_SOURCE_SYS_ORIGIN
         and trg.EVETP_VALID_TO = src.EVETP_VALID_TO WHERE (src.EVETP_SOURCE_ID is null);

        /* Close Old Records in Target */
        update  gap_catalog.ads_owner.EVENT_TYPES set
               EVETP_UPDATED_DATETIME = CURRENT_TIMESTAMP(),
               EVETP_UPDATE_PROCESS_KEY = CAST(p_process_key AS BIGINT),
               EVETP_CURRENT_FLAG = 'N', 
               EVETP_VALID_TO = DATEADD(DAY, -1, CAST(p_load_date AS DATE))
         where EVETP_CURRENT_FLAG = 'Y'
           and EVETP_VALID_TO = DATE('3000-01-01')
           and evetp_key||'.'||evetp_valid_from in (select tech_rid from gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_MAIN_ADS_MAP_SCD_DIFF where tech_rid is not null);

        /* Insert Changed Records */
        insert  into gap_catalog.ads_owner.EVENT_TYPES 
         ( EVETP_KEY, 
           EVETP_SOURCE_ID, 
           EVETP_SOURCE_SYSTEM_ID, 
           EVETP_SOURCE_SYS_ORIGIN, 
           EVETP_DESC, 
           EVETP_EVENT_TABLE_NAME, 
           EVETP_VALID_FROM, 
           EVETP_VALID_TO, 
           EVETP_CURRENT_FLAG, 
           EVETP_DELETED_FLAG, 
           EVETP_INSERTED_DATETIME, 
           EVETP_INSERT_PROCESS_KEY, 
           EVETP_UPDATED_DATETIME, 
           EVETP_UPDATE_PROCESS_KEY, 
           EVETP_TARGET, 
           EVETP_DATA_PATH)
        select EVETP_KEY, 
           EVETP_SOURCE_ID, 
           EVETP_SOURCE_SYSTEM_ID, 
           EVETP_SOURCE_SYS_ORIGIN, 
           EVETP_DESC, 
           EVETP_EVENT_TABLE_NAME, 
           CAST(p_load_date AS DATE) as EVETP_VALID_FROM, 
           DATE('3000-01-01') as EVETP_VALID_TO, 
           'Y' as EVETP_CURRENT_FLAG, 
           tech_del_flg as EVETP_DELETED_FLAG, 
           CURRENT_TIMESTAMP() as EVETP_INSERTED_DATETIME, 
           CAST(p_process_key AS BIGINT) as EVETP_INSERT_PROCESS_KEY, 
           CURRENT_TIMESTAMP() as EVETP_UPDATED_DATETIME, 
           CAST(p_process_key AS BIGINT) as EVETP_UPDATE_PROCESS_KEY, 
           EVETP_TARGET, 
           EVETP_DATA_PATH
          from gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_MAIN_ADS_MAP_SCD_DIFF
         where tech_new_rec = 'N';

        /* Insert New Records */
        insert  into gap_catalog.ads_owner.EVENT_TYPES 
         ( EVETP_KEY, 
           EVETP_SOURCE_ID, 
           EVETP_SOURCE_SYSTEM_ID, 
           EVETP_SOURCE_SYS_ORIGIN, 
           EVETP_DESC, 
           EVETP_EVENT_TABLE_NAME, 
           EVETP_VALID_FROM, 
           EVETP_VALID_TO, 
           EVETP_CURRENT_FLAG, 
           EVETP_DELETED_FLAG, 
           EVETP_INSERTED_DATETIME, 
           EVETP_INSERT_PROCESS_KEY, 
           EVETP_UPDATED_DATETIME, 
           EVETP_UPDATE_PROCESS_KEY, 
           EVETP_TARGET, 
           EVETP_DATA_PATH)
        select xxhash64(uuid()) AS EVETP_KEY,
           EVETP_SOURCE_ID,
           EVETP_SOURCE_SYSTEM_ID, 
           EVETP_SOURCE_SYS_ORIGIN, 
           EVETP_DESC, 
           EVETP_EVENT_TABLE_NAME, 
           CAST(p_load_date AS DATE) as EVETP_VALID_FROM, 
           DATE('3000-01-01') as EVETP_VALID_TO, 
           'Y' as EVETP_CURRENT_FLAG, 
           tech_del_flg as EVETP_DELETED_FLAG, 
           CURRENT_TIMESTAMP() as EVETP_INSERTED_DATETIME, 
           CAST(p_process_key AS BIGINT) as EVETP_INSERT_PROCESS_KEY, 
           CURRENT_TIMESTAMP() as EVETP_UPDATED_DATETIME, 
           CAST(p_process_key AS BIGINT) as EVETP_UPDATE_PROCESS_KEY, 
           EVETP_TARGET, 
           EVETP_DATA_PATH
          from gap_catalog.ads_etl_owner.DIFF_ADS_RDS_EVENT_TYPES_MAIN_ADS_MAP_SCD_DIFF
         where tech_new_rec = 'Y';

        CALL gap_catalog.log.complete_performance_test(
            plog_id => my_log_id
        );

    END;
    
END;